# Elementos do dataset

Antes mesmo de falar do projeto, ao longo do desenvolvimento, vamos explicitando qual o objetivo de cada versão de trabalho do arquivo principal.

O objetivo desta AvoidMort_ds4 é criar um dataset de mortalidades gerais para servir de comparação para a mortalidade evitável.

O dataset final deverá ter as seguintes características:


* setting: Brasil [poderemos encontrar um sufixo a Brasil que permita a sua subclassificação e carga deste dataset junto a outros]
* dimension: UF
* subgroup: municípios
* estimate: mortalidade por CIDs evitáveis e município
* setting-average: mortalidade por CIDs evitáveis por Brasil

# Projeto de Análise de Desigualdade em Saúde no modelo HEAT/OMS

## Bloco 2025/1 (mortalidade evitável)

Análise do sistema de mortalidade do SUS com agregação por unidades federativas

# Merge de dados de CID e População no arquivo principal

### Merge de dados de CID

In [7]:
ocde_evit_temp = pd.read_csv('downloads_outros/cid10_evit_ocde.csv')
ocde_evit_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   avoid_flag               346 non-null    object
 1   prevent_flag             189 non-null    object
 2   treat_flag               184 non-null    object
 3   Group                    346 non-null    object
 4   Causes of deaths         346 non-null    object
 5   Rationale for inclusion  346 non-null    object
 6   Range                    346 non-null    object
 7   cid_pai                  346 non-null    object
dtypes: object(8)
memory usage: 21.8+ KB


In [8]:
# Gerar nova tabela com valores únicos de CAUSABAS
dobr_evit_merged_temp = pd.read_parquet('files_clean/dobr_10_19a22.parquet')
causas_unicas = dobr_evit_merged_temp['CAUSABAS'].unique()

# Converter para DataFrame
df_causas_unicas = pd.DataFrame(causas_unicas, columns=['CAUSABAS'])

# Exibir as primeiras linhas do DataFrame
df_causas_unicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11895 entries, 0 to 11894
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CAUSABAS  11895 non-null  object
dtypes: object(1)
memory usage: 93.1+ KB


##########         Etapa diferenciada da versão          ##########

Utilizar 'Causes of deaths' de CID evitáveis e as demais como "Causa não evitável"

In [10]:
# Fazer merge da lista de causas_unicas com a lista de causas evitáveis

df_a = df_causas_unicas
column_a = 'CAUSABAS'
df_b = ocde_evit_temp
column_b = 'cid_pai'
column_return = 'Causes of deaths'



# Função para verificar a correspondência e retornar a classificação
def verificar_correspondencia(row):
    for index, row_b in df_b.iterrows():
        if row[column_a].startswith(row_b[column_b]):
            return row_b[column_return]
    return 'causa não evitável'

# Aplicando a função a cada linha do DataFrame A
df_a['causes_deaths'] = df_a.apply(verificar_correspondencia, axis=1)

print(tabulate(df_a.head()))

-  ----  ------------------------
0  I219  Ischaemic heart diseases
1  I10   Hypertensive diseases
2  X700  Intentional self-harm
3  C539  causa não evitável
4  V220  Transport Accidents
-  ----  ------------------------


In [11]:
# Merge dobr_merged_pop_clean and df_a by CAUSABAS
dobr_evit_merged_temp01 = pd.merge(dobr_evit_merged_temp, df_a, on='CAUSABAS', how='left')

# mostrar dados gerais da tabela após transformações
tab = dobr_evit_merged_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 4446922
+----+------------+-------------+-------------+--------------------------+
|    | CAUSABAS   | CODMUNRES   | ANO_OBITO   | causes_deaths            |
|----+------------+-------------+-------------+--------------------------|
|  0 | I219       | _292740     | _2019       | Ischaemic heart diseases |
|  1 | I10        | _355100     | _2019       | Hypertensive diseases    |
|  2 | X700       | _500630     | _2019       | Intentional self-harm    |
|  3 | C539       | _500500     | _2019       | causa não evitável       |
|  4 | V220       | _500627     | _2019       | Transport Accidents      |
+----+------------+-------------+-------------+--------------------------+


In [12]:
# Mostrar resultados de classificacao
dobr_evit_merged_temp01.groupby('causes_deaths').count()

,CAUSABAS,CODMUNRES,ANO_OBITO
causes_deaths,,,
Abdominal hernia,5886,5886,5886
Abscess of lung and mediastinum pyothorax,2512,2512,2512
Accidental Injuries,101139,101139,101139
Acute lower respiratory infections,3261,3261,3261
Acute pancreatitis,11838,11838,11838
...,...,...,...
Varicella,270,270,270
Venous thromboembolism,24558,24558,24558
Viral Hepatitis,8692,8692,8692


########## Etapa diferenciada da versão ##########

Não serão excluídos os registros com CID de doenças não evitáveis

In [ ]:
# # Manter na base só dados de CID evitáveis
# dobr_evit_merged_temp02 = dobr_evit_merged_temp01[dobr_evit_merged_temp01['classificacao']=='Avoidable mortality']

# # mostrar dados gerais da tabela após transformações
# tab = dobr_evit_merged_temp02
# print(f'Registro da tabela : {tab.shape[0]}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

#### Resultados do merge de cid_avoid e dobr

##########       Etapa diferenciada da versão      ##########

Atenção para etapa de salvar arquivo, para não sobrescrever outra versão

In [13]:
# Salvar arquivo final de DOBR

dobr_evit_merged_temp01.to_parquet('files__temp/AvoidMort_v4_dobr_evit_merged.parquet')

In [14]:
# Contador

shape_origin = dobr_evit_merged_temp.CAUSABAS.count()
shape_filt = dobr_evit_merged_temp01.CAUSABAS.count()
dif = shape_origin - shape_filt


print_y(f" O número de óbitos após a limpeza e que constituía a base de dados de DO era de {shape_origin} óbitos ")
print_y(f" O número de registros excluídos por apresentar CIDs NÃO EVITÁVEIS foi de {dif} óbitos ")
print_y(f" O número de óbitos por CID EVITÁVEIS é de {shape_filt} óbitos ")

 ==>  O número de óbitos após a limpeza e que constituía a base de dados de DO era de 4446922 óbitos 
 ==>  O número de registros excluídos por apresentar CIDs NÃO EVITÁVEIS foi de 0 óbitos 
 ==>  O número de óbitos por CID EVITÁVEIS é de 4446922 óbitos 


### Merge de dados de Municípios ao arquivo principal do SIM

Serão utilizados os arquivos:

* população: 'files_clean/censo_pop_clean.csv'

* sim: 'files_clean/dobr_clean.csv'

Como a população está clivada em Município, Idade (previamente, todos abaixo de 75 anos), Sexo e Raça, estes campos serão utilizados de maneira conjunta para realizar o merge.

In [15]:
# Verificar colunas de população no arquivo censo_pop_clean
censo_pop_mun_4merge_temp = pd.read_parquet('files__temp/censo_pop_mun.parquet')

# mostrar dados gerais da tabela após transformações
tab = censo_pop_mun_4merge_temp
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 11140
+----+-------+---------+------+---------------+----------------------------+-------------+---------------+
|    |   Ano | uf_co   | UF   | mun_cod_sim   | mun_no                     |   pop_total |   pop_under74 |
|----+-------+---------+------+---------------+----------------------------+-------------+---------------|
|  0 |  2010 | _11     | RO   | _110001       | Alta Floresta D'Oeste (RO) |       24392 |         23948 |
|  1 |  2010 | _11     | RO   | _110002       | Ariquemes (RO)             |       90353 |         89037 |
|  2 |  2010 | _11     | RO   | _110003       | Cabixi (RO)                |        6313 |          6180 |
|  3 |  2010 | _11     | RO   | _110004       | Cacoal (RO)                |       78574 |         76947 |
|  4 |  2010 | _11     | RO   | _110005       | Cerejeiras (RO)            |       17029 |         16646 |
+----+-------+---------+------+---------------+----------------------------+-------------+---------------+


In [16]:
# Criar um campo string que concatene ANO, CODMUN_SIM, Sexo e cor
censo_pop_mun_4merge_temp01 = censo_pop_mun_4merge_temp.copy()

censo_pop_mun_4merge_temp01['POP_MERGE'] = '_' + censo_pop_mun_4merge_temp01['Ano'].astype(str) + censo_pop_mun_4merge_temp01['mun_cod_sim']


# mostrar dados gerais da tabela após transformações
tab = censo_pop_mun_4merge_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 11140
+----+-------+---------+------+---------------+----------------------------+-------------+---------------+--------------+
|    |   Ano | uf_co   | UF   | mun_cod_sim   | mun_no                     |   pop_total |   pop_under74 | POP_MERGE    |
|----+-------+---------+------+---------------+----------------------------+-------------+---------------+--------------|
|  0 |  2010 | _11     | RO   | _110001       | Alta Floresta D'Oeste (RO) |       24392 |         23948 | _2010_110001 |
|  1 |  2010 | _11     | RO   | _110002       | Ariquemes (RO)             |       90353 |         89037 | _2010_110002 |
|  2 |  2010 | _11     | RO   | _110003       | Cabixi (RO)                |        6313 |          6180 | _2010_110003 |
|  3 |  2010 | _11     | RO   | _110004       | Cacoal (RO)                |       78574 |         76947 | _2010_110004 |
|  4 |  2010 | _11     | RO   | _110005       | Cerejeiras (RO)            |       17029 |         16646 | _2010_110005

In [17]:
# Contar quantos valores são repetidos na coluna 'POP_MERGE'
repeated_values = censo_pop_mun_4merge_temp01['POP_MERGE'].value_counts()

# Filtrar os valores que aparecem mais de uma vez
repeated_more_than_once = repeated_values[repeated_values > 1]

# Exibir os valores repetidos
print(repeated_more_than_once)

Series([], Name: count, dtype: int64)


In [18]:
# Exibir arquivo dobr_clean
dobr_merged_temp = pd.read_parquet('files__temp/AvoidMort_v4_dobr_evit_merged.parquet')

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 4446922
+----+------------+-------------+-------------+--------------------------+
|    | CAUSABAS   | CODMUNRES   | ANO_OBITO   | causes_deaths            |
|----+------------+-------------+-------------+--------------------------|
|  0 | I219       | _292740     | _2019       | Ischaemic heart diseases |
|  1 | I10        | _355100     | _2019       | Hypertensive diseases    |
|  2 | X700       | _500630     | _2019       | Intentional self-harm    |
|  3 | C539       | _500500     | _2019       | causa não evitável       |
|  4 | V220       | _500627     | _2019       | Transport Accidents      |
+----+------------+-------------+-------------+--------------------------+


In [37]:
# # Gerar um campo de ano para proxy de 2022 (para ser aplicados aos anos de 19 a 22)
dobr_merged_temp01 = dobr_merged_temp.copy()

dobr_merged_temp01['ANO_PROXY'] = dobr_merged_temp01['ANO_OBITO'].apply(lambda x: '_2010' if x == '_2010' else '_2022')

# # Criar um campo string que concatene ANO, CODMUNRES
dobr_merged_temp01['POP_MERGE'] = dobr_merged_temp01['ANO_PROXY'].astype(str) + dobr_merged_temp['CODMUNRES']

# # mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 4446922
+----+------------+-------------+-------------+--------------------------+--------------+-------------+
|    | CAUSABAS   | CODMUNRES   | ANO_OBITO   | causes_deaths            | POP_MERGE    | ANO_PROXY   |
|----+------------+-------------+-------------+--------------------------+--------------+-------------|
|  0 | I219       | _292740     | _2019       | Ischaemic heart diseases | _2022_292740 | _2022       |
|  1 | I10        | _355100     | _2019       | Hypertensive diseases    | _2022_355100 | _2022       |
|  2 | X700       | _500630     | _2019       | Intentional self-harm    | _2022_500630 | _2022       |
|  3 | C539       | _500500     | _2019       | causa não evitável       | _2022_500500 | _2022       |
|  4 | V220       | _500627     | _2019       | Transport Accidents      | _2022_500627 | _2022       |
+----+------------+-------------+-------------+--------------------------+--------------+-------------+


In [44]:
# # Verificar operação acima
print(f'Anos de ANO_OBITO {dobr_merged_temp01['ANO_OBITO'].unique()}')
print(f'Anos de ANO_PROXY {dobr_merged_temp01['ANO_PROXY'].unique()}')
print(f'Anos de Censo {censo_pop_mun_4merge_temp01['Ano'].unique()}')

Anos de ANO_OBITO ['_2019' '_2020' '_2021' '_2022' '_2010']
Anos de ANO_PROXY ['_2022' '_2010']
Anos de Censo ['2010' '2022']


In [43]:
# Contar o número de caracteres de cada valor na coluna 'POP_MERGE'
lengths = dobr_merged_temp01['POP_MERGE'].str.len()
num_records = lengths[lengths > 12].count()
print(f"Número de registros de dobr com 'POP_MERGE' maior que 12 caracteres: {num_records}")
lengths2 = censo_pop_mun_4merge_temp01['POP_MERGE'].str.len()
num_records2 = lengths2[lengths2 > 12].count()
print(f"Número de registros de censo_pop_mun com 'POP_MERGE' maior que 12 caracteres: {num_records2}")

Número de registros de dobr com 'POP_MERGE' maior que 12 caracteres: 0
Número de registros de censo_pop_mun com 'POP_MERGE' maior que 12 caracteres: 0


##########       Etapa diferenciada da versão      ##########

Iremos excluir o CAUSABAS, mas agrupar por município, ano e cause_deaths.
O contador de mortes portanto está atrelado ao cause_deaths.

In [57]:
# Agrupar por POP_MERGE e somar os registros, não contabilizando mais por CAUSABAS

col_group= [
            # 'CAUSABAS', 
            'CODMUNRES', 'ANO_OBITO','POP_MERGE','causes_deaths']
dobr_merged_temp02 = dobr_merged_temp01.groupby(col_group)['CAUSABAS'].count().reset_index()

# Renomear o campo CAUSABAS para CONTADOR
dobr_merged_temp02.rename(columns={'CAUSABAS':'CONTADOR'}, inplace=True)


# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp02
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 473555
Total de óbitos : 4446922
+----+-------------+-------------+--------------+--------------------------------------------------------------------+------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    | causes_deaths                                                      |   CONTADOR |
|----+-------------+-------------+--------------+--------------------------------------------------------------------+------------|
|  0 | _110000     | _2010       | _2010_110000 | Accidental Injuries                                                |          3 |
|  1 | _110000     | _2010       | _2010_110000 | Assault                                                            |         10 |
|  2 | _110000     | _2010       | _2010_110000 | Cerebrovascular diseases                                           |          1 |
|  3 | _110000     | _2010       | _2010_110000 | Chronic lower respiratory diseases                                 |          1 |
|  4 | _110000     | 

In [33]:
# Verificação dos nomes das colunas
censo_pop_mun_4merge_temp01.columns

Index(['Ano', 'uf_co', 'UF', 'mun_cod_sim', 'mun_no', 'pop_total',
       'pop_under74', 'POP_MERGE'],
      dtype='object')

In [58]:
# Definir campos de censo_pop para merge
censo_pop_mun_4merge_temp02 = censo_pop_mun_4merge_temp01[['POP_MERGE','pop_under74','UF','mun_no']]

# Realizar o merge dos dataframes
dobr_merged_temp03 = pd.merge(dobr_merged_temp02, censo_pop_mun_4merge_temp02, left_on=['POP_MERGE'], right_on=['POP_MERGE'], how='left')

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp03
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 473555
Total de óbitos : 4446922
+----+-------------+-------------+--------------+--------------------------------------------------------------------+------------+---------------+------+----------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    | causes_deaths                                                      |   CONTADOR |   pop_under74 |   UF |   mun_no |
|----+-------------+-------------+--------------+--------------------------------------------------------------------+------------+---------------+------+----------|
|  0 | _110000     | _2010       | _2010_110000 | Accidental Injuries                                                |          3 |           nan |  nan |      nan |
|  1 | _110000     | _2010       | _2010_110000 | Assault                                                            |         10 |           nan |  nan |      nan |
|  2 | _110000     | _2010       | _2010_110000 | Cerebrovascular diseases                                         

In [59]:
# Filtrar registros onde pop_under74 não é nulo
dobr_merged_temp04 = dobr_merged_temp03.dropna(subset=['pop_under74']).reset_index(drop=True)

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp04
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 472557
Total de óbitos : 4440843
+----+-------------+-------------+--------------+-------------------------------------------+------------+---------------+------+----------------------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    | causes_deaths                             |   CONTADOR |   pop_under74 | UF   | mun_no                     |
|----+-------------+-------------+--------------+-------------------------------------------+------------+---------------+------+----------------------------|
|  0 | _110001     | _2010       | _2010_110001 | Accidental Injuries                       |          1 |         23948 | RO   | Alta Floresta D'Oeste (RO) |
|  1 | _110001     | _2010       | _2010_110001 | Alcohol-specific disorders and poisonings |          2 |         23948 | RO   | Alta Floresta D'Oeste (RO) |
|  2 | _110001     | _2010       | _2010_110001 | Appendicitis                              |          1 |         23948 | RO   | Alta Floresta D'Oest

##### Arquivo tratado final

In [61]:
dobr_merged_temp04.to_parquet('files__temp/AvoidMort_v4_dobr_merged.parquet')

In [60]:
# Resultados

obito_origin = dobr_merged_temp.CAUSABAS.count()
obito_final = dobr_merged_temp04.CONTADOR.sum()
dif = obito_origin - obito_final


print_y(f" O número de óbitos após merge com tabela de CIDs evitáveis era de {obito_origin} óbitos ")
print_y(f" Foram excluídos, por apresentar problemas para carga de informações de município, {dif} óbitos ")
print_y(f" O número de óbitos total após filtro de CID evitáveis e inclusão de população do censo é de {obito_final} óbitos ")


# # Filtrar registros onde pop_under74 é null
# dobr_merged_pop_nulos = dobr_merged_pop[dobr_merged_pop['pop_under74'].isnull()]

# # Exibir os registros filtrados
# dobr_merged_pop_nulos['obito_count'].sum()

 ==>  O número de óbitos após merge com tabela de CIDs evitáveis era de 4446922 óbitos 
 ==>  Foram excluídos, por apresentar problemas para carga de informações de município, 6079 óbitos 
 ==>  O número de óbitos total após filtro de CID evitáveis e inclusão de população do censo é de 4440843 óbitos 


# Etapa 3 - Aplicação de Modelo do HEAT ao Bloco

O modelo HEAT apresenta um framework bem rígido e objetivo para carga de dados.

O dataset precisa estar organizado com os labels e de acordo com a definição presente no template.

E depois precisa ser validado em planilha excel disponibilizada pela organização.

Ajustes: com a identificação da população por sexo e raça, encontramos 37 registros com população 0. Tivemos que excluir estes registros (incluindo os óbitos) no momento da preparação do dataset do heat em razão do modelo não aceitar estes valores.

## Dataset 4 - BR/CIDs/uf/Mun


O primeiro dataset que iremos construir tem as seguintes características:

* set = Brasil
* indicador = causes_deaths
* dimen = UF
* subgroup = município
* estimate = mortalidade por ano, causa (indicador) e subgroup
* set-avg = mortalidade por ano e causa (indicador)

Estas escolhas implicam em podermos disponibilizar ao usuário do HEAT a possibilidade de ver os municípios (dimensões) de maneira agregada por UF. 


In [3]:
# Define the URL and the destination path
url = "https://srhdpeuwpubsa.blob.core.windows.net/whdh/HIDR/heat_plus_template_validation.xlsm"
destination_path = "downloads_outros/heat_plus_template_validation.xlsm"

# Download the file
response = requests.get(url)
with open(destination_path, 'wb') as file:
    file.write(response.content)

# Note: Enabling macros programmatically is not recommended due to security risks.
# Macros should be enabled manually in Excel settings.

In [3]:
# Abrir arquivo final de DO após tratamento
heatds_temp = pd.read_parquet('files__temp/AvoidMort_v4_dobr_merged.parquet')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 472557
Total de óbitos : 4440843
+----+-------------+-------------+--------------+-------------------------------------------+------------+---------------+------+----------------------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    | causes_deaths                             |   CONTADOR |   pop_under74 | UF   | mun_no                     |
|----+-------------+-------------+--------------+-------------------------------------------+------------+---------------+------+----------------------------|
|  0 | _110001     | _2010       | _2010_110001 | Accidental Injuries                       |          1 |         23948 | RO   | Alta Floresta D'Oeste (RO) |
|  1 | _110001     | _2010       | _2010_110001 | Alcohol-specific disorders and poisonings |          2 |         23948 | RO   | Alta Floresta D'Oeste (RO) |
|  2 | _110001     | _2010       | _2010_110001 | Appendicitis                              |          1 |         23948 | RO   | Alta Floresta D'Oest

In [38]:
# # Agregar os dados sem CID e gerar colunas de soma de obito_count e pop_under74
# heat_semCID = heat_.groupby(['UF', 'mun_nome', 'CODMUNRES', 'SEXO', 'RACACOR', 'ANO']).agg({
#     'obito_count': 'sum',
#     'pop_under74': 'sum'
# }).reset_index()

##########       Etapa diferenciada da versão      ##########

Para melhor registro, utilizamos o label mort_ind_mun_causedeath para deixar claro que o indicador é relativo a município e causa de morte

In [4]:
# Gerar o indicador de mortalidade (por 100 mil habitantes) e exibir as primeiras linhas do DataFrame
heatds_temp01 = heatds_temp.copy()

heatds_temp01['mort_ind_mun_causedeath'] = (heatds_temp01['CONTADOR']/heatds_temp01['pop_under74'])*100000

# mostrar dados gerais da tabela após transformações
tab = heatds_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 472557
Total de óbitos : 4440843
+----+-------------+-------------+--------------+-------------------------------------------+------------+---------------+------+----------------------------+---------------------------+
|    | CODMUNRES   | ANO_OBITO   | POP_MERGE    | causes_deaths                             |   CONTADOR |   pop_under74 | UF   | mun_no                     |   mort_ind_mun_causedeath |
|----+-------------+-------------+--------------+-------------------------------------------+------------+---------------+------+----------------------------+---------------------------|
|  0 | _110001     | _2010       | _2010_110001 | Accidental Injuries                       |          1 |         23948 | RO   | Alta Floresta D'Oeste (RO) |                   4.17571 |
|  1 | _110001     | _2010       | _2010_110001 | Alcohol-specific disorders and poisonings |          2 |         23948 | RO   | Alta Floresta D'Oeste (RO) |                   8.35143 |
|  2 | _11

In [5]:
# Contar e gerar tabela de indicador de mortalidade infinitos [inf]
registros_inf = heatds_temp01[np.isinf(heatds_temp01['mort_ind_mun_causedeath'])]
inf_count = np.isinf(heatds_temp01['mort_ind_mun_causedeath']).sum()
nulos_mort_ind_mun = heatds_temp01['mort_ind_mun_causedeath'].isnull().sum()


# Exibir os registros filtrados
print(f"Número de valores infinitos em mort_ind_mun: {inf_count}")
print(f"Valores nulos em 'mort_ind_mun': {nulos_mort_ind_mun}")



# Exportar registros de indicador de mortalidade infinitos [inf]
# registros_inf.to_csv('exportar/relatorio_reg_mort_ind_inf.csv')

Número de valores infinitos em mort_ind_mun: 0
Valores nulos em 'mort_ind_mun': 0


In [ ]:
# # Limpar a base de registros de indicador de mortalidade infinitos [inf]
# heat_ds_treat2 = heat_ds_treat.copy()

# # Substituir valores infinitos por NaN
# heat_ds_treat2['mort_ind_mun'].replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
# # Rodar se for necessário excluir valores nulos
# heatds_temp02 = heatds_temp01.copy()

# # Excluir registros com NaN na coluna mort_ind_mun
# heatds_temp02.dropna(subset=['mort_ind_mun'], inplace=True)

# # Mudar o nome da coluna mort_ind para mort_ind_mun
# # heat_ds_treat2.rename(columns={'mort_ind':'mort_ind_mun'}, inplace=True)


# # Verificar as primeiras linhas do DataFrame atualizado
# heat_ds_treat1.head()

In [6]:
# Verificar se existe valor infinito ou negativo na coluna mort_ind_mun
heatds_temp01.describe()

,CONTADOR,pop_under74,mort_ind_mun_causedeath
count,472557.000000,4.725570e+05,472557.000000
mean,9.397476,8.285577e+04,26.700906
std,117.927717,3.874791e+05,49.167856
min,1.000000,7.820000e+02,0.009177
25%,1.000000,9.544000e+03,4.990269
50%,2.000000,1.975300e+04,12.059817
75%,5.000000,4.994000e+04,26.271166
max,35890.000000,1.089676e+07,870.195794


##### Agrupar por UF para gerar setting e setting-average

De acordo com o template do HEAT, o setting average: "must be consistent for the same combinantion of: setting, year, source and indicator".

Isto implica que os valores para cada setting (UF) não traz interferência nem da dimensão (município) e nem do subgrupo (sexo-raça-cor).

In [ ]:
# # Agrupar por 'UF', 'ANO', e calcular a média de 'mort_ind'
# dobr_grouped_uf = heat_ds_treat2.groupby(['UF',
#                                              'ANO_OBITO',
#                                             #  'SEXO',
#                                             #  'RACACOR'
#                                              ]).agg({'CONTADOR': 'sum',
#                                                     'pop_under74': 'sum',
#                                                     'mort_ind_mun': 'mean'
#                                                 }).reset_index()
# dobr_grouped_uf.rename(columns={'mort_ind_mun': 'mort_ind_uf',
#                                 'CONTADOR': 'obito_count_uf',
#                                 'pop_under74': 'pop_under74_uf',
#                                 }, inplace=True)


# # Exibir as primeiras linhas do DataFrame resultante
# dobr_grouped_uf.head()

In [ ]:
# dobr_grouped_uf.describe()

In [ ]:
# Salvar arquivo por UF para exportação
# dobr_grouped_uf.to_excel('exportar/dobr_grouped_uf_v2.xlsx', index=False)

In [ ]:
# heat_ds_treat2.columns

In [ ]:
# # Trazer dados por UF (setting e setting_average)

# # Realizar o merge entre heat_ e uf_sett
# heat_treat00 = pd.merge(heat_ds_treat2, dobr_grouped_uf, on=['UF',
#                                                                 'ANO_OBITO',
#                                                                 # 'SEXO',
#                                                                 # 'RACACOR'
#                                                                 ], how='left')

# # Exibir as primeiras linhas do dataframe resultante
# heat_treat00.head()

In [ ]:
# # Teste de aplicação de setting_average (médias por UF)
# filtered_data = heat_treat00[   (heat_treat00['ANO_OBITO'] == '_2022') &
#                                 (heat_treat00['UF'] == 'AC') 
#                                 # & (heat_treat00['SEXO'] == '_1') &
#                                 #(heat_treat00['RACACOR'] == '_4')
#                               ]
# filtered_data.head()

In [ ]:
# heat_treat00.columns

In [ ]:
# heat_treat00.pop_under74_uf.sum()

In [ ]:
# obito_num = heat_treat00.CONTADOR.sum()
# pop_under74 = heat_treat00.pop_under74.sum()

# print(f"\033[33m ==> Número de mortes: {obito_num} \033[0m")
# print(f"\033[33m ==> Número de população abaixo de 75 anos é de {pop_under74} \033[0m")  


In [ ]:
# heat_template = row
# heat_template

##### Criar colunas do dataset

##########       Etapa diferenciada da versão      ##########

Esta etapa precisa bastante atenção.

Se considerarmos causes_deaths como dimensão, ela precisa ser ignorada na criação so set-avg.

Pelo contrário, se formos tratar causes_deaths como indicador, ele precisa ser considerado na criação do set-avg.

Vamos realizar abaixo causes_deaths como indicador.

In [7]:
heatds_temp01.columns

Index(['CODMUNRES', 'ANO_OBITO', 'POP_MERGE', 'causes_deaths', 'CONTADOR',
       'pop_under74', 'UF', 'mun_no', 'mort_ind_mun_causedeath'],
      dtype='object')

In [8]:
# Na média ponderada de mortalidade evitável, pegamos os valores de mortalidade para cada município e dividimos pela população, para conseguir o valor para o Brasil e por ano

# Função para calcular a média ponderada
def weighted_avg(group):
    return np.average(group['mort_ind_mun_causedeath'], weights=group['pop_under74'])

# Agrupar os dados pelo ano e aplicar a função de média ponderada
weighted_avg_per_year = heatds_temp01.groupby(['ANO_OBITO','causes_deaths']).apply(weighted_avg).reset_index()

# Renomear as colunas para melhor entendimento
weighted_avg_per_year.columns = ['ANO_OBITO','causes_deaths','set_avg_BR']

# Exibir o resultado
print(weighted_avg_per_year)

    ANO_OBITO                              causes_deaths  set_avg_BR
0       _2010                           Abdominal hernia    0.903132
1       _2010  Abscess of lung and mediastinum pyothorax    0.673743
2       _2010                        Accidental Injuries   11.020345
3       _2010         Acute lower respiratory infections    0.579934
4       _2010                         Acute pancreatitis    1.916919
..        ...                                        ...         ...
393     _2022               Upper respiratory infections    0.653949
394     _2022                                  Varicella    0.409434
395     _2022                     Venous thromboembolism    4.069222
396     _2022                            Viral Hepatitis    1.232339
397     _2022                         causa não evitável  175.117448

[398 rows x 3 columns]


/tmp/ipykernel_4253/1189972450.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_avg_per_year = heatds_temp01.groupby(['ANO_OBITO','causes_deaths']).apply(weighted_avg).reset_index()


In [39]:
# Realizar o merge entre heat_ds_treat2 e grouped_data usando a coluna ANO_OBITO
heatds_temp02 = pd.merge(heatds_temp01, weighted_avg_per_year, on=['ANO_OBITO','causes_deaths'], how='left')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp02
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
tab.head()
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 472557
Total de óbitos : 4440843


,CODMUNRES,ANO_OBITO,POP_MERGE,causes_deaths,CONTADOR,pop_under74,UF,mun_no,mort_ind_mun_causedeath,set_avg_BR
0,_110001,_2010,_2010_110001,Accidental Injuries,1,23948.0,RO,Alta Floresta D'Oeste (RO),4.175714,11.020345
1,_110001,_2010,_2010_110001,Alcohol-specific disorders and poisonings,2,23948.0,RO,Alta Floresta D'Oeste (RO),8.351428,10.215044
2,_110001,_2010,_2010_110001,Appendicitis,1,23948.0,RO,Alta Floresta D'Oeste (RO),4.175714,0.759074
3,_110001,_2010,_2010_110001,Assault,7,23948.0,RO,Alta Floresta D'Oeste (RO),29.229998,29.188749
4,_110001,_2010,_2010_110001,Cerebrovascular diseases,3,23948.0,RO,Alta Floresta D'Oeste (RO),12.527142,27.555941


##########       Etapa diferenciada da versão      ##########

Para esta versão
* set = Brasil
* indicador = causes_deaths
* dimen = UF
* subgroup = município
* estimate = mortalidade por ano, causa (indicador) e subgroup
* set-avg = mortalidade por ano e causa (indicador)

In [23]:
# Etapas para gerar indicator_abbr exclusivos

# Etapa 1 - Gerar uma tabela apenas com valores exclusivos de heatds_temp02.causes_deaths, em ordem crescente
unique_causes_deaths = heatds_temp02[['causes_deaths']].drop_duplicates().sort_values(by='causes_deaths').reset_index(drop=True)

# Etapa 2 - Criar coluna indicator_abbr com os primeiros 5 caracteres de causes_deaths
unique_causes_deaths['indicator_abbr'] = unique_causes_deaths['causes_deaths'].str[:5]

# Etapa 3 - Para aqueles indicator_abbr que tiverem o mesmo nome, gerar sufixo incremental a partir de 1
unique_causes_deaths['indicator_abbr'] = unique_causes_deaths.groupby('indicator_abbr').cumcount().astype(str).radd(unique_causes_deaths['indicator_abbr'])

# Etapa 3.1 - excluir valores que terminam com 0
unique_causes_deaths['indicator_abbr'] = unique_causes_deaths['indicator_abbr'].str.replace('0', '')

# Exibir a tabela resultante
unique_causes_deaths

,causes_deaths,indicator_abbr
0,Abdominal hernia,Abdom
1,Abscess of lung and mediastinum pyothorax,Absce
2,Accidental Injuries,Accid
3,Acute lower respiratory infections,Acute
4,Acute pancreatitis,Acute1
...,...,...
76,Varicella,Varic
77,Venous thromboembolism,Venou
78,Viral Hepatitis,Viral
79,Whooping cough,Whoop


In [40]:
# Fazer merge com informações de indicator_abbr exclusivos
heatds_temp03 = pd.merge(heatds_temp02,unique_causes_deaths,on='causes_deaths', how='left')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp03
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))
tab.head()

Registros da tabela : 472557
Total de óbitos : 4440843


,CODMUNRES,ANO_OBITO,POP_MERGE,causes_deaths,CONTADOR,pop_under74,UF,mun_no,mort_ind_mun_causedeath,set_avg_BR,indicator_abbr
0,_110001,_2010,_2010_110001,Accidental Injuries,1,23948.0,RO,Alta Floresta D'Oeste (RO),4.175714,11.020345,Accid
1,_110001,_2010,_2010_110001,Alcohol-specific disorders and poisonings,2,23948.0,RO,Alta Floresta D'Oeste (RO),8.351428,10.215044,Alcoh
2,_110001,_2010,_2010_110001,Appendicitis,1,23948.0,RO,Alta Floresta D'Oeste (RO),4.175714,0.759074,Appen
3,_110001,_2010,_2010_110001,Assault,7,23948.0,RO,Alta Floresta D'Oeste (RO),29.229998,29.188749,Assau
4,_110001,_2010,_2010_110001,Cerebrovascular diseases,3,23948.0,RO,Alta Floresta D'Oeste (RO),12.527142,27.555941,Cereb


In [41]:
# Gerar colunas sem informação ou com informação padronizada

# heat_treat01 = heat_semCID_clean # caso não seja agrupado por UF anteriormente
heatds_temp04 = heatds_temp03.copy() # caso seja agrupado por UF anteriormente

heatds_temp04['setting'] = 'Brasil'
# heat_treat01['setting_average'] = set_avg_BR

# heatds_temp03['indicator_abbr'] = 'AVM'
# heatds_temp03['indicator_abbr'] = heatds_temp03['causes_deaths'].str.slice(0, 5)
# heatds_temp03['indicator_name'] = 'Avoidable mortality (deaths per 100 000 population)'
heatds_temp04['indicator_name'] = heatds_temp04['causes_deaths'] + '(deaths per 100 000 population)'
heatds_temp04['source'] = 'SIM/SUS & Censo/IBGE'
heatds_temp04['favourable_indicator'] = 0
heatds_temp04['indicator_scale'] = 100000
heatds_temp04['ordered_dimension'] = 0
heatds_temp04['subgroup_order'] = 0
heatds_temp04['reference_subgroup'] = 0
heatds_temp04['note'] = ''
heatds_temp04['se'] = ''
heatds_temp04['ci_lb'] = ''
heatds_temp04['ci_ub'] = ''
heatds_temp04['iso3'] = 'BRA'

# Renomear colunas de heat_treat01
heatds_temp04 = heatds_temp04.rename(columns={
    'ANO_OBITO':'date',
    'pop_under74':'population',
    'UF':'dimension',
    'mun_no':'subgroup',
    # 'mort_ind_mun':'estimate',
    'mort_ind_mun_causedeath':'estimate',
    'set_avg_BR':'setting_average',
    # 'mort_ind_uf':'setting_average'
})

# Ajustar coluna de ano
heatds_temp04['date'] = heatds_temp04['date'].str.replace('_','').astype(int)


# mostrar dados gerais da tabela após transformações
tab = heatds_temp04
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))
tab.head()


Registros da tabela : 472557
Total de óbitos : 4440843


,CODMUNRES,date,POP_MERGE,causes_deaths,CONTADOR,population,dimension,subgroup,estimate,setting_average,...,favourable_indicator,indicator_scale,ordered_dimension,subgroup_order,reference_subgroup,note,se,ci_lb,ci_ub,iso3
0,_110001,2010,_2010_110001,Accidental Injuries,1,23948.0,RO,Alta Floresta D'Oeste (RO),4.175714,11.020345,...,0,100000,0,0,0,,,,,BRA
1,_110001,2010,_2010_110001,Alcohol-specific disorders and poisonings,2,23948.0,RO,Alta Floresta D'Oeste (RO),8.351428,10.215044,...,0,100000,0,0,0,,,,,BRA
2,_110001,2010,_2010_110001,Appendicitis,1,23948.0,RO,Alta Floresta D'Oeste (RO),4.175714,0.759074,...,0,100000,0,0,0,,,,,BRA
3,_110001,2010,_2010_110001,Assault,7,23948.0,RO,Alta Floresta D'Oeste (RO),29.229998,29.188749,...,0,100000,0,0,0,,,,,BRA
4,_110001,2010,_2010_110001,Cerebrovascular diseases,3,23948.0,RO,Alta Floresta D'Oeste (RO),12.527142,27.555941,...,0,100000,0,0,0,,,,,BRA


In [285]:
# # Trazer labels para sexo e raçacor
# heat_treat02 = heat_treat01.copy()

# heat_treat02['SEXO'] = heat_treat02['SEXO'].map({
#                                             '_1':'Homens',
#                                             '_2':'Mulheres'
#                                             })
# heat_treat02['RACACOR'] = heat_treat02['RACACOR'].map({
#                                             '_1':'Branca',
#                                             '_2':'Preta',
#                                             '_3':'Amarela',
#                                             '_4':'Parda',
#                                             '_5':'Indígena',
#                                             '_9':'Sem declaração'
#                                             })

# heat_treat02['subgroup'] = heat_treat02['SEXO'] + ' - raça/cor '  +  heat_treat02['RACACOR']


In [ ]:
# heat_treat02['subgroup'].unique()

In [ ]:
# heat_treat02.columns

In [43]:
# Define the order of columns
column_order = [
    'setting',
    'date',
    'source',
    'indicator_abbr',
    'indicator_name',
    'dimension',
    'subgroup',
    'estimate',
    'se',
    'ci_lb',
    'ci_ub',
    'population',
    'note',
    'setting_average',
    'iso3',
    'favourable_indicator',
    'indicator_scale',
    'ordered_dimension',
    'subgroup_order',
    'reference_subgroup'
    ]  


# Reorder the columns in the DataFrame
heatds_temp05 = heatds_temp04[column_order]


In [46]:
# Transformar estimate e setting_average em int
heatds_temp06 = heatds_temp05.copy()

heatds_temp06['estimate'] = heatds_temp06['estimate'].astype(int)
heatds_temp06['setting_average'] = heatds_temp06['setting_average'].astype(int)

# mostrar dados gerais da tabela após transformações
tab = heatds_temp06
print(f'Registros da tabela : {tab.shape[0]}')
# print(f'Total de óbitos : {tab.CONTADOR.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))
tab.head()

Registros da tabela : 472557


,setting,date,source,indicator_abbr,indicator_name,dimension,subgroup,estimate,se,ci_lb,ci_ub,population,note,setting_average,iso3,favourable_indicator,indicator_scale,ordered_dimension,subgroup_order,reference_subgroup
0,Brasil,2010,SIM/SUS & Censo/IBGE,Accid,Accidental Injuries(deaths per 100 000 populat...,RO,Alta Floresta D'Oeste (RO),4,,,,23948.0,,11,BRA,0,100000,0,0,0
1,Brasil,2010,SIM/SUS & Censo/IBGE,Alcoh,Alcohol-specific disorders and poisonings(deat...,RO,Alta Floresta D'Oeste (RO),8,,,,23948.0,,10,BRA,0,100000,0,0,0
2,Brasil,2010,SIM/SUS & Censo/IBGE,Appen,Appendicitis(deaths per 100 000 population),RO,Alta Floresta D'Oeste (RO),4,,,,23948.0,,0,BRA,0,100000,0,0,0
3,Brasil,2010,SIM/SUS & Censo/IBGE,Assau,Assault(deaths per 100 000 population),RO,Alta Floresta D'Oeste (RO),29,,,,23948.0,,29,BRA,0,100000,0,0,0
4,Brasil,2010,SIM/SUS & Censo/IBGE,Cereb,Cerebrovascular diseases(deaths per 100 000 po...,RO,Alta Floresta D'Oeste (RO),12,,,,23948.0,,27,BRA,0,100000,0,0,0


##########       Etapa diferenciada da versão      ##########

Atenção da hora de salvar o arquivo, com as informações corretas de versão

In [47]:
### Arquivo pronto para exportação
heatds_temp06.to_excel('files_out/AvoidMort_v4_ds_4val.xlsx', index=False)

In [95]:
# Gerar arquivo dataset diretamente no arquivo de validação do HEAT

# Carregar o arquivo original
workbook = openpyxl.load_workbook('files_out/heat_plus_template_validation.xlsm', keep_vba=True)

# Apagar a aba 'template'
if 'template' in workbook.sheetnames:
    del workbook['template']

# Carregar o arquivo que será copiado como nova aba
workbook_to_copy = openpyxl.load_workbook('files_out/AvoidMort_v4_ds_4val.xlsx')
sheet_to_copy = workbook_to_copy.active

# Copiar a aba para o workbook original
new_sheet = workbook.create_sheet('template')

for row in sheet_to_copy.iter_rows(values_only=True):
    new_sheet.append(row)

# Salvar o arquivo com o novo nome
workbook.save('files_out/AvoidMort_v4_heat_plus_template_ds_4val.xlsm')

# Etapa 4 - Reunião de alinhamento - ajustes

# Etapa 5 - Preparação para publicação de resultados